# Quantitative metrics for image-patches

In [1]:
import os
import sys
import csv

from scipy import ndimage as nd
from skimage import measure



current_path = os.path.abspath('.')
root_path = os.path.dirname(os.path.dirname(current_path))
sys.path.append(root_path)

from sourcecode.OCDC.ocdc_dataloader import *
from sourcecode.wsi_image_utils import *
from sourcecode.evaluation_utils import *



#dataset_dir = "../../datasets/OCDC"
dataset_dir = "/media/dalifreire/DADOS/PhD/github/tumor_regions_segmentation/datasets/OCDC"

batch_size = 1
patch_size = (640, 640)
color_model = "LAB"
dataloaders = create_dataloader(tile_size="{}x{}".format(patch_size[0], patch_size[1]),
                                batch_size=batch_size, 
                                shuffle=False,
                                img_input_size=patch_size,
                                img_output_size=patch_size,
                                dataset_dir=dataset_dir,
                                color_model=color_model)

dataset_train_size = len(dataloaders['train'].dataset)
dataset_test_size = len(dataloaders['test'].dataset)
print("-")

tile_size = 20
magnification=0.625

threshold_prob = 0.50
threshold_itc = 200/(0.243 * pow(2, 5))

wsi_images_dir_normal = "{}/testing/normal/wsi".format(dataset_dir)
wsi_images_dir_tumor = "{}/testing/tumor/wsi".format(dataset_dir)

trained_model_version = "OCDC__Size-640x640_Epoch-400_Images-840_Batch-1__random_9_operations"
results_dir="{}/results/{}/testing".format(dataset_dir, trained_model_version)
csv_file_path = "{}/ocdc_quantitative_analysis_{}.csv".format(results_dir, threshold_prob)

wsi_tissue_patches = {}
with open(csv_file_path, mode='w') as medidas_file:

    medidas_writer = csv.writer(medidas_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    medidas_writer.writerow(['wsi_image', 'patch_image', 'class', 'accuracy', 'precision', 'f1/dice', 'jaccard', 'sensitivity/recall', 'specificity', 'pixels', 'tp', 'tn', 'fp', 'fn'])

    for batch_idx, (data, target, fname, original_size) in enumerate(dataloaders['test']):

        # wsi image number
        wsi_image_number = fname[0].split("_")[0]
        if wsi_image_number not in wsi_tissue_patches:
            
            # extract the tissue region from original image and draw the heat grid
            wsi_image_path = "{}/{}.svs".format(wsi_images_dir_tumor, wsi_image_number)
            if not os.path.exists(wsi_image_path):
                wsi_image_path = "{}/{}.svs".format(wsi_images_dir_normal, wsi_image_number)
            
             # scale down image
            wsi_image = open_wsi(wsi_image_path)
            pil_scaled_down_image = scale_down_wsi(wsi_image, magnification, False)
            np_scaled_down_image = pil_to_np(pil_scaled_down_image)

            # extract tissue region 
            np_tissue_mask, np_masked_image = extract_normal_region_from_wsi(wsi_image_path, np_scaled_down_image, None)
            pil_masked_image = np_to_pil(np_masked_image)

            # draw the heat grid
            pil_img_result, heat_grid, number_of_tiles = draw_heat_grid(np_masked_image, tile_size)

            tissue_patches = []
            for idx, (position, row, column, location, size, color) in enumerate(heat_grid):
                if color != GREEN_COLOR: 
                    tissue_patches.append("{}_r{}c{}.png".format(wsi_image_number, row, column))

            wsi_tissue_patches[wsi_image_number] = tissue_patches
            #print(wsi_tissue_patches)

        # check if the patch was excluded in preprocessing step
        patch_excludde_in_preprocessing = fname[0] not in wsi_tissue_patches[wsi_image_number]

        # load the mask image
        mask_np_img = target[0].numpy()

        # roi x non_roi classes
        wsi_class = "tumor" if wsi_image_path.find("tumor") > 0 else "normal"
        patch_class = "roi" if np.max(np.unique(mask_np_img)) > 0 else 'non_roi'
                

        # load the predicted image result
        patch_results_dir = "{}/{}/patch/640x640/{}".format(results_dir, wsi_class, wsi_image_number)
        unet_result_img = "{}/01-unet_result/{}".format(patch_results_dir, fname[0])
        predicted_pil_img = Image.fromarray(np.zeros(mask_np_img.shape)) if patch_excludde_in_preprocessing else load_pil_image(unet_result_img, gray=True) if os.path.isfile(unet_result_img) else Image.fromarray(np.zeros(mask_np_img.shape))
        predicted_np_img = np.copy(pil_to_np(predicted_pil_img))
        predicted_np_img = predicted_np_img * (1.0/255)
        predicted_np_img = basic_threshold(predicted_np_img, threshold=threshold_prob, output_type="uint8")

        predicted_labels = measure.label(predicted_np_img, connectivity=2)
        predicted_np_img = np.zeros((predicted_np_img.shape[0], predicted_np_img.shape[1]))
        labels = np.unique(predicted_labels)
        properties = measure.regionprops(predicted_labels)
        for lbl in range(1, np.max(labels)):
            major_axis_length = properties[lbl-1].major_axis_length
            if major_axis_length > threshold_itc:
                predicted_np_img[predicted_labels == lbl] = 1


        # metrics
        auc = 0.0 #roc_auc_score(mask_np_img, predicted_np_img)
        precision = precision_score(mask_np_img, predicted_np_img)
        recall = recall_score(mask_np_img, predicted_np_img)
        accuracy = accuracy_score(mask_np_img, predicted_np_img)
        f1 = f1_score(mask_np_img, predicted_np_img)
        specificity = specificity_score(mask_np_img, predicted_np_img)
        jaccard = jaccard_score(mask_np_img, predicted_np_img)

        total_pixels, tn, fp, fn, tp = tn_fp_fn_tp(mask_np_img, predicted_np_img)

        print("Results for {:26} ({:7} - {:8} - {:04.2f} accuracy)".format(fname[0], patch_class, "excluded" if patch_excludde_in_preprocessing else "unet", accuracy))
        #print("   Precision: \t{}".format(precision))
        #print("   Recall/Sen: \t{}".format(recall))
        #print("   F1/Dice: \t{}".format(f1))
        #print("   Accuracy: \t{}".format(accuracy))
        #print("   Specificity: {}".format(specificity))
        #print("   Jaccard: \t{}".format(jaccard))
        #print("   TP = {} TN = {} FP = {} FN = {}".format(tp, tn, fp, fn))
        #print("-")

        medidas_writer.writerow([wsi_image_number, fname[0], patch_class, accuracy, precision, f1, jaccard, recall, specificity, total_pixels, tp, tn, fp, fn])


2021-10-23 11:38:17,193 :: INFO load_dataset :: [training] /media/dalifreire/DADOS/PhD/github/tumor_regions_segmentation/datasets/OCDC/training
2021-10-23 11:38:17,242 :: INFO load_dataset :: [training] /media/dalifreire/DADOS/PhD/github/tumor_regions_segmentation/datasets/OCDC/training
2021-10-23 11:38:17,290 :: INFO load_dataset :: [testing] /media/dalifreire/DADOS/PhD/github/tumor_regions_segmentation/datasets/OCDC/testing
2021-10-23 11:38:17,337 :: INFO create_dataloader :: Train images (640x640): 840 augmentation: random
2021-10-23 11:38:17,338 :: INFO create_dataloader :: Test images (640x640): 180 augmentation: no_augmentation
2021-10-23 11:38:17,486 :: INFO transform :: Epoch: '1' augmentation no_augmentation None


-


2021-10-23 11:38:17,958 :: INFO extract_normal_region_from_wsi :: 	 Extracting normal regions from wsi image: '1009009.svs'


Results for 1009009_r27c10.png         (roi     - unet     - 0.95 accuracy)
Results for 1009009_r27c5.png          (roi     - unet     - 0.94 accuracy)
Results for 1009009_r28c12.png         (roi     - unet     - 0.87 accuracy)


/home/dalifreire/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

Results for 1009009_r10c2.png          (non_roi - excluded - 1.00 accuracy)
Results for 1009009_r10c4.png          (non_roi - excluded - 1.00 accuracy)
Results for 1009009_r11c2.png          (non_roi - excluded - 1.00 accuracy)
Results for 1009009_r15c4.png          (non_roi - unet     - 1.00 accuracy)
Results for 1009009_r16c16.png         (non_roi - excluded - 1.00 accuracy)
Results for 1009009_r20c2.png          (non_roi - unet     - 1.00 accuracy)
Results for 1009009_r20c22.png         (non_roi - unet     - 1.00 accuracy)
Results for 1009009_r21c7.png          (non_roi - unet     - 1.00 accuracy)
Results for 1009009_r2c3.png           (non_roi - unet     - 1.00 accuracy)
Results for 1009009_r30c22.png         (non_roi - excluded - 1.00 accuracy)
Results for 1009009_r34c13.png         (non_roi - unet     - 1.00 accuracy)
Results for 1009009_r38c17.png         (non_roi - excluded - 1.00 accuracy)
Results for 1009009_r3c2.png           (non_roi - unet     - 1.00 accuracy)
Results for 

2021-10-23 11:39:46,303 :: INFO extract_normal_region_from_wsi :: 	 Extracting normal regions from wsi image: '1009010x1000902.svs'


Results for 1009010x1000902_r20c33.png (roi     - unet     - 0.95 accuracy)
Results for 1009010x1000902_r22c35.png (roi     - unet     - 0.99 accuracy)
Results for 1009010x1000902_r23c32.png (roi     - unet     - 0.96 accuracy)
Results for 1009010x1000902_r24c33.png (roi     - unet     - 0.95 accuracy)
Results for 1009010x1000902_r25c34.png (roi     - unet     - 0.04 accuracy)
Results for 1009010x1000902_r26c29.png (roi     - unet     - 0.93 accuracy)
Results for 1009010x1000902_r31c26.png (roi     - unet     - 0.97 accuracy)
Results for 1009010x1000902_r35c31.png (roi     - unet     - 0.95 accuracy)
Results for 1009010x1000902_r37c2.png  (roi     - unet     - 0.87 accuracy)
Results for 1009010x1000902_r42c5.png  (roi     - unet     - 0.70 accuracy)
Results for 1009010x1000902_r18c13.png (non_roi - unet     - 1.00 accuracy)
Results for 1009010x1000902_r21c0.png  (non_roi - unet     - 1.00 accuracy)
Results for 1009010x1000902_r22c31.png (non_roi - unet     - 1.00 accuracy)
Results for 

2021-10-23 11:41:12,648 :: INFO extract_normal_region_from_wsi :: 	 Extracting normal regions from wsi image: '1009011.svs'


Results for 1009011_r13c12.png         (roi     - unet     - 0.96 accuracy)
Results for 1009011_r14c10.png         (roi     - unet     - 0.94 accuracy)
Results for 1009011_r14c11.png         (roi     - unet     - 0.95 accuracy)
Results for 1009011_r14c8.png          (roi     - unet     - 0.95 accuracy)
Results for 1009011_r16c8.png          (roi     - unet     - 0.98 accuracy)
Results for 1009011_r24c14.png         (roi     - unet     - 0.95 accuracy)
Results for 1009011_r25c5.png          (roi     - unet     - 0.89 accuracy)
Results for 1009011_r33c11.png         (roi     - unet     - 0.87 accuracy)
Results for 1009011_r34c12.png         (roi     - unet     - 0.96 accuracy)
Results for 1009011_r6c12.png          (roi     - unet     - 0.93 accuracy)
Results for 1009011_r0c53.png          (non_roi - excluded - 1.00 accuracy)
Results for 1009011_r10c6.png          (non_roi - unet     - 1.00 accuracy)
Results for 1009011_r11c18.png         (non_roi - unet     - 1.00 accuracy)
Results for 